In [35]:
# Libraries

import os
from automathon import NFA
from collections import deque
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
from rouge import Rouge
import json

import sumy

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import seaborn as sns
import matplotlib.pyplot as plt

## Using sumy

[sumy](https://github.com/miso-belica/sumy) is a python library that implements
TextRank and other algorithms to summarize text.

Sumy implements TextRank algorithm based on the paper [TextRank: Bringing Order into Texts](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

### Import dataset

In [36]:
df = pd.read_excel('data/business_articles.xlsx')

In [37]:
df.head()

,Article,Summary
0,UK economy facing 'major risks'\n\nThe UK manu...,"""Despite some positive news for the export sec..."
1,Aids and climate top Davos agenda\n\nClimate c...,"At the same time, about 100,000 people are exp..."
2,Asian quake hits European shares\n\nShares in ...,The unfolding scale of the disaster in south A...
3,India power shares jump on debut\n\nShares in ...,"Shares in India's largest power producer, Nati..."
4,Lacroix label bought by US firm\n\nLuxury good...,LVMH said the French designer's haute couture ...


In [38]:
text = df["Article"][0]

In [39]:
LANGUAGE = "english"

In [40]:
sentences = sent_tokenize(text)

In [41]:
stopwords_eng = stopwords.words(LANGUAGE)

In [42]:
sentences_clean = [re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]

In [43]:
sentence_tokens = [[words for words in sentence.split() if words not in stopwords_eng] for sentence in sentences_clean]

In [44]:
graph: dict[str, list[str]] = {}
graph_inv: dict[str, list[str]] = {}
in_degree: dict[str, int] = {node: 0 for sentence in sentence_tokens for node in sentence}
out_degree: dict[str, int] = {node: 0 for sentence in sentence_tokens for node in sentence}

In [45]:
for sentence in sentence_tokens:
  for i in range(len(sentence)-1):
    word1 = sentence[i]
    word2 = sentence[i+1]
    
    in_degree[word2] += 1
    out_degree[word1] += 1
    
    # Build graph
    if word1 not in graph:
      graph[word1] = [word2]
    elif word2 not in graph[word1]:
      graph[word1].append(word2)
    
    # Build inverse graph
    if word2 not in graph_inv:
      graph_inv[word2] = [word1]
    elif word1 not in graph_inv[word2]:
      graph_inv[word2].append(word1)

In [46]:
pretty_graph = json.dumps(graph, indent=4)

In [47]:
print(pretty_graph)

{
    "uk": [
        "economy",
        "manufacturing"
    ],
    "economy": [
        "facing",
        "still"
    ],
    "facing": [
        "major"
    ],
    "major": [
        "risks",
        "concern"
    ],
    "risks": [
        "uk",
        "warned"
    ],
    "manufacturing": [
        "sector",
        "also",
        "bcc",
        "service"
    ],
    "sector": [
        "continue",
        "worrying",
        "uncertain"
    ],
    "continue": [
        "face"
    ],
    "face": [
        "serious"
    ],
    "serious": [
        "challenges",
        "problems"
    ],
    "challenges": [
        "next"
    ],
    "next": [
        "two",
        "1218"
    ],
    "two": [
        "years"
    ],
    "years": [
        "british"
    ],
    "british": [
        "chamber"
    ],
    "chamber": [
        "commerce"
    ],
    "commerce": [
        "bcc"
    ],
    "bcc": [
        "said",
        "found",
        "noted",
        "director"
    ],
    "groups": [
       

In [48]:
def draw_graph(graph: dict[str, list[str]], f_set=None, image_name="graph_connections") -> None:
  Q :set[str] = set()
  sigma = {'-'}
  delta: dict[str, dict[str, set[str]]] = {}

  for node in graph:
    adj_set = set(graph[node])
    Q.add(node)
    Q = Q.union(adj_set)
    
    delta[node] = {'-': adj_set}
    # for i in range(len(sentence)-1):
    #   word1 = sentence[i]
    #   word2 = sentence[i+1]
      
    #   Q.add(word1)
    #   Q.add(word2)
      
    #   if word1 not in delta:
    #     delta[word1] = {'-': {word2}}
    #   else:
    #     delta[word1]['-'].add(word2)

  initialState = sentence_tokens[0][0]
  if f_set is None:
    F = {}
  else:
    F = f_set

  automata = NFA(Q, sigma, delta, initialState, F)
  automata.view(image_name)
  
  ## Delete .gv file
  if os.path.isfile(f"{image_name}.gv"):
    os.remove(f"{image_name}.gv")
  
  ## Rename .gv.png to .png
  if os.path.isfile(f"{image_name}.gv.png"):
    os.rename(f"{image_name}.gv.png", f"{image_name}.png")

In [49]:
# draw_graph(sentence_tokens)

# Finding articulation points in the graph

In [52]:
# Implementation adapted from: https://cp-algorithms.com/graph/cutpoints.html#implementation
timer = 0

def find_cutpoints(graph: dict[str, list[str]]) -> set[str]:
  global timer
  visited: dict[str, bool] = {}
  tin: dict[str, int] = {}
  low: dict[str, int] = {}
  timer = 0
  cut_points: set[str] = set()
  
  def dfs(v: str, p: str = "") -> None:
    global timer
    visited[v] = True
    tin[v] = low[v] = timer
    timer += 1
    children_count = 0
    
    if v not in graph:
      return
    
    for to in graph[v]:
      if to == p:
        continue
      if visited[to]:
        low[v] = min(low[v], tin[to])
      else:
        dfs(to, v)
        low[v] = min(low[v], low[to])
        if low[v] >= tin[v] and p != "":
          cut_points.add(v)
        children_count += 1
    
    if p == "" and children_count > 1:
      cut_points.add(v)
  
  for node in graph:
    visited[node] = False
    tin[node] = -1
    low[node] = -1
    for adj in graph[node]:
      visited[adj] = False
      tin[adj] = -1
      low[adj] = -1
      
  for v in graph:
    if not visited[v]:
      dfs(v)
  
  return cut_points

In [53]:
cut_points = find_cutpoints(graph=graph)

In [54]:
cut_points

{'2005',
 '25',
 'bcc',
 'inability',
 'little',
 'persistent',
 'pick',
 'sectors',
 'slowdown',
 'strongly',
 'sufficiently',
 'sustain'}

In [55]:
draw_graph(graph=graph, f_set=cut_points, image_name="graph_connections_cutpoints")

In [56]:
for cut_point in cut_points:
  print(f"Cut point: {cut_point} - in_degree: {in_degree[cut_point]} - out_degree: {out_degree[cut_point]}")

Cut point: pick - in_degree: 1 - out_degree: 1
Cut point: bcc - in_degree: 5 - out_degree: 6
Cut point: inability - in_degree: 1 - out_degree: 1
Cut point: sufficiently - in_degree: 1 - out_degree: 1
Cut point: little - in_degree: 1 - out_degree: 1
Cut point: 25 - in_degree: 1 - out_degree: 1
Cut point: persistent - in_degree: 1 - out_degree: 1
Cut point: sectors - in_degree: 2 - out_degree: 2
Cut point: sustain - in_degree: 1 - out_degree: 1
Cut point: 2005 - in_degree: 1 - out_degree: 1
Cut point: slowdown - in_degree: 1 - out_degree: 1
Cut point: strongly - in_degree: 1 - out_degree: 1


# Prune graph

In [57]:
def prune_graph(
  graph: dict[str, list[str]],
  graph_inv: dict[str, list[str]],
  in_degree: dict[str, int],
  out_degree: dict[str, int],
) -> dict[str, list[str]]:
    q = []
    
    for node in out_degree:
      if out_degree[node] == 0:
        q.append(node)
    
    
    while len(q) > 0:
      node = q.pop(0)
      out_degree[node] = -1
      
      if node not in graph_inv:
        continue
      
      for next_node in graph_inv[node]:
        out_degree[next_node] -= 1
        if out_degree[next_node] == 0:
          q.append(next_node)
    
    new_graph: dict[str, list[str]] = {}
    
    for node in graph:
      if out_degree[node] == -1:
        continue
      
      new_graph[node] = []
      for next_node in graph[node]:
        if out_degree[next_node] != -1:
          new_graph[node].append(next_node)
    
    return new_graph

In [58]:
pruned_graph = prune_graph(graph, graph_inv, in_degree, out_degree)
pruned_cut_points = find_cutpoints(pruned_graph)

In [59]:
pruned_cut_points

set()

In [60]:
draw_graph(graph=pruned_graph, f_set=pruned_cut_points, image_name="graph_connections_pruned")

In [61]:
text = df["Article"][0]

In [62]:
sentences = sent_tokenize(text)

Cleaning the sentences

In [63]:
sentences_clean = [re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stopwords_eng = stopwords.words('english')

In [64]:
sentence_tokens = [[words for words in sentence.split() if words not in stopwords_eng] for sentence in sentences_clean]

In [65]:
w2v = Word2Vec(sentence_tokens,min_count=1)

In [66]:
sentence_embeddings = [ [w2v.wv[word][0] for word in words] for words in sentence_tokens ]
max_len = max([len(tokens) for tokens in sentence_tokens])

sentence_embeddings = [np.pad(embedding, (0, max_len-len(embedding)), 'constant') for embedding in sentence_embeddings]

In [67]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])

for i, row_embedding in enumerate(sentence_embeddings):
  for j, column_embedding in enumerate(sentence_embeddings):
    similarity_matrix[i][j] = 1-spatial.distance.cosine(row_embedding, column_embedding)

In [72]:
graph: dict[str, list[str]] = {}

In [73]:
for s1 in sentence_tokens:
  sent1 = ' '.join(s1)
  for s2 in sentence_tokens:
    sent2 = ' '.join(s2)
    
    if sent1 not in graph:
      graph[sent1] = [sent2]
    else:
      graph[sent1].append(sent2)
    
    if sent2 not in graph:
      graph[sent2] = [sent1]
    else:
      graph[sent2].append(sent1)

In [ ]:
def draw_graph(graph: dict[str, list[str]], weights=None, f_set=None, image_name="graph_connections") -> None:
  Q :set[str] = set()
  sigma = {'-'}
  delta: dict[str, dict[str, set[str]]] = {}

  for node in graph:
    adj_set = set(graph[node])
    Q.add(node)
    Q = Q.union(adj_set)
    
    delta[node] = {'-': adj_set}
    # for i in range(len(sentence)-1):
    #   word1 = sentence[i]
    #   word2 = sentence[i+1]
      
    #   Q.add(word1)
    #   Q.add(word2)
      
    #   if word1 not in delta:
    #     delta[word1] = {'-': {word2}}
    #   else:
    #     delta[word1]['-'].add(word2)

  initialState = sentence_tokens[0][0]
  if f_set is None:
    F = {}
  else:
    F = f_set

  automata = NFA(Q, sigma, delta, initialState, F)
  automata.view(image_name)
  
  ## Delete .gv file
  if os.path.isfile(f"{image_name}.gv"):
    os.remove(f"{image_name}.gv")
  
  ## Rename .gv.png to .png
  if os.path.isfile(f"{image_name}.gv.png"):
    os.rename(f"{image_name}.gv.png", f"{image_name}.png")